# GDAL 101

<a target="_blank" href="https://colab.research.google.com/github/mapninja/GDAL-101/blob/main/GDAL-101.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Introduction to GDAL/OGR Workshop

Welcome to the GDAL/OGR workshop! In this workshop, we will explore the powerful capabilities of the Geospatial Data Abstraction Library (GDAL) and the OGR Simple Features Library (part of GDAL) for working with geospatial data. GDAL/OGR is an open-source library that provides tools for reading, writing, and manipulating raster and vector geospatial data formats.

### What is GDAL/OGR?

- **GDAL (Geospatial Data Abstraction Library)**: A library for reading and writing raster geospatial data formats. It supports a wide range of formats, including GeoTIFF, JPEG, PNG, and more.
- **OGR (part of GDAL)**: A library for reading and writing vector geospatial data formats. It supports formats such as Shapefile, GeoJSON, KML, and more.

### Workshop Datasets

In this workshop, we will work with the following datasets:
1. **Vector Data**:
    - `zipcodes.shp`: A shapefile containing zip code boundaries.
    - `pools.shp`: A shapefile containing pool locations.
2. **Raster Data**:
    - `doqq.tif`: A raster dataset representing an orthophoto.
    - `dem10m.dem`: A Digital Elevation Model (DEM) with 10-meter resolution.

### Accessing GDAL/OGR in Colab

To ensure that everyone has a consistent environment and similar outputs, we will use the `!<command>` syntax to run GDAL/OGR commands directly in the Jupyter Notebook. This approach allows us to leverage the full power of GDAL/OGR without relying on Python bindings, making it easier to follow along and reproduce the results.


## Overview of steps:

1. Installation: Install the GDAL library in your COnda or Colab Environment    
2. Verification: Verify GDAL installation.
3. Download the data: Use Python to download and unzip the data
4. Check out the data in `leafmap``
5. OGR for Vector Data: Use GDAL to inspect vector data.
6. GDAL for Raster Data: Use GDAL to inspect raster data.
7. Converting Data: Convert data formats using GDAL.
8. Reprojecting Data: Reproject data using GDAL.
9. Querying Data: Query data using SQL with GDAL.
10. Clipping to create new data: Clip data using GDAL.
11. Fun with DEMs: Creating derived datasets from your Digital ELevation Model

# Installation

This workshop assumes that you are using Colab, or have already installed GDAL in your local Conda Environment. 
Install necessary packages

### For Colab, use this install block

If you are using Colab, use this code block to install the required software on the Kernel you are using. 

In [ ]:
%%capture # Hide output
if 'google.colab' in str(get_ipython()):  # Check if the code is running in Google Colab
  !apt-get install gdal-bin  # Install GDAL binaries

### For all users 

Use this block to make sure all required python packages are installed. 

In [ ]:
!pip install gdal folium leafmap matplotlib rasterio geopandas jupyter-server-proxy xarray localtileserver

### Check your installation of GDAL

In [ ]:
!gdalinfo --version  # This command checks the installed version of GDAL

### Imports

These are the necessary Python Libraries for this workshop.

In [ ]:
# from osgeo import gdal, ogr  # Import the gdal and ogr modules from the osgeo package
import zipfile  # Import the zipfile module for handling zip files
import os  # Import the os module for interacting with the operating system
import geopandas as gpd  # Import the geopandas module for working with geospatial data
import leafmap  # Import the leafmap module for creating interactive maps with geospatial data
import matplotlib.pyplot as plt  # Import the matplotlib.pyplot module for plotting
import rasterio  # Import the rasterio module for working with raster data


## Download and Unzip the data in Colab

This assumes you are working in [Colab](https://colab.research.google.com/). First, you will need to place the `GDAL_101_data.zip` file in the `./contents` folder on Colab. This is the folder that is visible by default, in your Files Panel, in Colab.

If you need to download and load the data manually, for some reason, you can find it at:
https://drive.google.com/file/d/18c1L3nt3FTcvd2YLm5HwXzfiulAgs0en/view


## Downloading and Unzipping the Dataset in Python

In this section, we will walk through the process of downloading a dataset from a URL and unzipping it using Python. This is a common task in data processing workflows, especially when working with large datasets that are distributed as compressed files.

### Steps to Download and Unzip the Dataset

1. **Download the Dataset**:
    - We use the `requests` library to download the dataset from a specified URL.
    - The dataset is saved to a local file path.

2. **Unzip the Dataset**:
    - We use the `zipfile` library to unzip the downloaded file.
    - The contents of the zip file are extracted to a specified directory.

### Code Explanation

#### Downloading the Dataset

The following code snippet demonstrates how to download the dataset using the `requests` library:

In [ ]:
import requests

# URL of the dataset
url = "https://drive.usercontent.google.com/download?id=18c1L3nt3FTcvd2YLm5HwXzfiulAgs0en&export=download&authuser=0&confirm=t&uuid=b210a738-a88d-4278-ad6c-9ff8a9b205b6&at=AN_67v1XxwHdrXmMIItrqla7n7IG%3A1730219939009"

# Path where the dataset will be saved
output_path = './GDAL_101_data.zip'

# Download the dataset
response = requests.get(url)
with open(output_path, 'wb') as file:
    file.write(response.content)

print(f"Dataset downloaded and saved to {output_path}")

#### Unzip the dataset to your ./data/ folder

In [ ]:
# Define the path to the zip file and the extraction directory (in Colab, right-click and select Copy Path to get this location)
zip_file_path = './GDAL_101_data.zip'
extract_to_path = './data'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Data extracted to {os.path.abspath(extract_to_path)}")

## Check out the data in Leafmap

### Use leafmap to visualize the zipcodes.shp (vector dataset)

In [ ]:
# Load the shapefile using geopandas
gdf = gpd.read_file(f"{extract_to_path}/zipcodes/zipcodes.shp")

# Create a map centered around the data
m = leafmap.Map(center=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom=9)

# Add the shapefile layer to the map
m.add_gdf(gdf, layer_name="Zipcodes")

# Display the map
m


### Use leafmap to visualize the doqq.tif (raster dataset)

In [ ]:
# Create a map centered around the approximate location of the raster data
doqq_map_center = [30.2672, -97.7431]  # Coordinates for Austin, TX
doqq_map = leafmap.Map(center=doqq_map_center, zoom=10)

# Define the path to the doqq.tif file
doqq_tif_path = f"{extract_to_path}/doqq.tif"

# Add the raster layer to the map
doqq_map.add_raster(doqq_tif_path, layer_name="DOQQ")

# Display the map
doqq_map

## OGR for Vector Data

The Geospatial Data Abstraction Library (GDAL) is a powerful open-source library for reading, writing, and manipulating geospatial data. One of the key utilities of GDAL is its ability to extract detailed metadata from various spatial datasets, including both vector and raster formats. This metadata provides essential information about the dataset, such as its coordinate system, extent, resolution, and attribute information.

### Key Benefits of Using GDAL for Metadata Extraction

1. **Comprehensive Metadata**: GDAL can extract a wide range of metadata, including spatial reference information, data type, dimensions, and more. This comprehensive metadata is crucial for understanding the characteristics and quality of the dataset.

2. **Support for Multiple Formats**: GDAL supports a vast array of geospatial data formats, making it a versatile tool for metadata extraction across different types of datasets, such as shapefiles, GeoTIFFs, and more.

3. **Command-Line and Programmatic Access**: GDAL provides both command-line utilities (e.g., `gdalinfo`, `ogrinfo`) and programmatic interfaces (e.g., Python bindings) for extracting metadata. This flexibility allows users to integrate metadata extraction into automated workflows and scripts.

4. **Standardized Output**: The metadata extracted by GDAL is presented in a standardized format, making it easier to parse and utilize in subsequent data processing and analysis tasks.

5. **Efficiency**: GDAL is optimized for performance, enabling efficient extraction of metadata even for large and complex datasets.

### Common GDAL Commands for Metadata Extraction

- **`gdalinfo`**: This command is used to extract metadata from raster datasets. It provides detailed information about the raster, including its size, coordinate system, geotransform, and more.
- **`ogrinfo`**: This command is used to extract metadata from vector datasets. It provides information about the layers, attributes, and spatial reference of the vector data.

By leveraging GDAL's capabilities for metadata extraction, users can gain valuable insights into their spatial datasets, facilitating better data management, analysis, and decision-making.

### Use `ogrinfo` to get information about your vector datasets

In [ ]:
!ogrinfo ./data/pools/pools.shp                        # Display information about the shapefile 'pools.shp'

### Explanation of the Output

The output message provides information about the successful opening of a shapefile using the GDAL library. Let's break down the message:

1. **INFO:** This indicates that the message is informational. It is not an error or a warning, but rather a confirmation that an operation has been successfully completed.

2. **Open of `./data/pools/pools.shp'**: This part specifies the file path of the shapefile that has been opened. In this case, the shapefile is located at `./data/pools/pools.shp`.

3. **using driver `ESRI Shapefile' successful.**: This indicates that the file was successfully opened using the `ESRI Shapefile` driver. GDAL supports various drivers for different geospatial data formats, and in this case, it used the driver specific to ESRI Shapefiles.

4. **1: pools (3D Polygon)**: This part provides information about the contents of the shapefile. It indicates that the shapefile contains one layer named `pools`, and the geometries in this layer are 3D polygons.


In [ ]:
!ogrinfo ./data/zipcodes/zipcodes.shp                  # Display information about the shapefile 'zipcodes.shp'

Here's a brief explanation of the output:

- `INFO: Open of './data/zipcodes/zipcodes.shp' using driver 'ESRI Shapefile' successful.`: This line indicates that GDAL successfully opened the shapefile located at 

`./data/zipcodes.shp`

 using the ESRI Shapefile driver.
- `1: zipcodes (Polygon)`: This line indicates that the shapefile contains one layer named `zipcodes`, and the geometries in this layer are of type `Polygon`.



### Use `ogrinfo` to get information about the layer in your zipcodes.shp vector dataset

In [ ]:
!ogrinfo -so ./data/zipcodes/zipcodes.shp zipcodes     # Display summary information about the 'zipcodes' layer in the 'zipcodes.shp' shapefile

The output provides detailed information about the shapefile `zipcodes.shp` located in the `./data` directory. Here's a breakdown of the output:

1. **Opening `zipcodes.shp`:**
    ```
    INFO: Open of `./data/zipcodes/zipcodes.shp' using driver `ESRI Shapefile' successful.
    1: zipcodes (Polygon)
    ```
    - **INFO:** Indicates an informational message.
    - **Open of `./data/zipcodes/zipcodes.shp'**: Specifies the file path of the shapefile that has been opened.
    - **using driver `ESRI Shapefile' successful.**: Indicates that the file was successfully opened using the ESRI Shapefile driver.
    - **1: zipcodes (Polygon)**: Indicates that the shapefile contains one layer named `zipcodes`, and the geometries in this layer are polygons.

2. **Detailed Information about `zipcodes` Layer:**
    ```
    Layer name: zipcodes
    Geometry: Polygon
    Feature Count: 80
    Extent: (-98.589809, 29.669673) - (-97.184067, 31.037805)
    Layer SRS WKT:
    GEOGCRS["WGS 84",
         DATUM["World Geodetic System 1984",
              ELLIPSOID["WGS 84",6378137,298.257223563,
                    LENGTHUNIT["metre",1]]],
         PRIMEM["Greenwich",0,
              ANGLEUNIT["degree",0.0174532925199433]],
         CS[ellipsoidal,2],
              AXIS["latitude",north,
                    ORDER[1],
                    ANGLEUNIT["degree",0.0174532925199433]],
              AXIS["longitude",east,
                    ORDER[2],
                    ANGLEUNIT["degree",0.0174532925199433]],
         ID["EPSG",4326]]
    Data axis to CRS axis mapping: 2,1
    ZIPCODE: String (12.0)
    NAME: String (30.0)
    CREATED_DA: Date
    CREATED_BY: String (50.0)
    MODIFIED_D: Date
    MODIFIED_B: String (50.0)
    SHAPE_AREA: Real (19.11)
    SHAPE_LEN: Real (19.11)
    ```
    - **Layer name:** The name of the layer is `zipcodes`.
    - **Geometry:** The geometries in this layer are polygons.
    - **Feature Count:** There are 80 features in this layer.
    - **Extent:** The spatial extent of the layer is from coordinates (-98.589809, 29.669673) to (-97.184067, 31.037805).
    - **Layer SRS WKT:** The spatial reference system (SRS) is WGS 84, with EPSG code 4326.
    - **Attributes:**
      - **ZIPCODE:** String with a maximum length of 12 characters.
      - **NAME:** String with a maximum length of 30 characters.
      - **CREATED_DA:** Date field.
      - **CREATED_BY:** String with a maximum length of 50 characters.
      - **MODIFIED_D:** Date field.
      - **MODIFIED_B:** String with a maximum length of 50 characters.
      - **SHAPE_AREA:** Real number with precision 19.11.
      - **SHAPE_LEN:** Real number with precision 19.11.


## GDAL for Raster Data

## Utility of Using GDAL for Working with Raster Data

GDAL provides a suite of utilities and libraries that facilitate the manipulation, transformation, and analysis of raster datasets. Here are some key benefits and utilities of using GDAL for raster data:

### Key Benefits

1. **Support for Multiple Formats**: GDAL supports a wide range of raster formats, including GeoTIFF, JPEG, PNG, and more. This versatility allows users to work with various data sources seamlessly.

2. **Efficient Data Processing**: GDAL is optimized for performance, enabling efficient processing of large raster datasets. This is crucial for handling high-resolution satellite imagery and other extensive geospatial data.

3. **Comprehensive Metadata Extraction**: GDAL can extract detailed metadata from raster datasets, such as coordinate systems, resolution, and extent. This information is vital for understanding the characteristics and quality of the data.

4. **Reprojection and Transformation**: GDAL provides tools for reprojecting and transforming raster data to different coordinate systems. This is essential for integrating datasets from various sources with different spatial references.

5. **Data Conversion**: GDAL allows for the conversion of raster data between different formats. This is useful for preparing data for specific applications or software that require particular formats.

6. **Clipping and Subsetting**: GDAL can clip and subset raster data based on specified boundaries or windows. This functionality is useful for focusing analysis on specific areas of interest.

7. **Raster Calculations**: GDAL supports various raster calculations, such as generating hillshades, slope maps, and color reliefs. These derived products are valuable for visualizing and analyzing terrain and other raster data.

### Common GDAL Utilities for Raster Data

- **`gdalinfo`**: Extracts metadata from raster datasets, providing detailed information about the file.
- **`gdal_translate`**: Converts raster data between different formats and performs subsetting and resampling.
- **`gdalwarp`**: Reprojects and transforms raster data to different coordinate systems.
- **`gdaldem`**: Generates derived products such as hillshades, slope maps, and color reliefs from Digital Elevation Models (DEMs).
- **`gdal_contour`**: Creates contour lines from raster elevation data.

By leveraging GDAL's powerful capabilities, users can efficiently manage, analyze, and visualize raster data, making it an indispensable tool in the geospatial domain.

### Use `gdalinfo` to get information about the dem10m.dem raster dataset

In [ ]:
# Use the gdalinfo command to get information about the DEM (Digital Elevation Model) file
# The DEM file is located in the 'dem10m' directory within the 'data' directory
!gdalinfo ./data/dem10m/dem10m.dem


### Explanation of `gdalinfo` Output for `dem10m.dem` Raster Dataset

The output of the `gdalinfo` command provides detailed metadata about the `dem10m.dem` raster dataset. Here's a breakdown of the key components:

1. **Driver**:
    ```
    Driver: USGSDEM/USGS Optional ASCII DEM (and CDED)
    ```
    - The driver used to read the file is `USGSDEM`, which supports USGS Optional ASCII DEM and CDED formats.

2. **Files**:
    ```
    Files: ./data/dem10m/dem10m.dem
             ./data/dem10m/dem10m.dem.aux.xml
    ```
    - The primary file is `dem10m.dem`.
    - An auxiliary XML file `dem10m.dem.aux.xml` is also present, which may contain additional metadata.

3. **Size**:
    ```
    Size is 3600, 3600
    ```
    - The raster size is 3600 pixels by 3600 pixels.

4. **Coordinate System**:
    ```
    Coordinate System is:
    GEOGCRS["NAD83",
         DATUM["North American Datum 1983",
              ELLIPSOID["GRS 1980",6378137,298.257222101,
                    LENGTHUNIT["metre",1]]],
         PRIMEM["Greenwich",0,
              ANGLEUNIT["degree",0.0174532925199433]],
         CS[ellipsoidal,2],
              AXIS["geodetic latitude (Lat)",north,
                    ORDER[1],
                    ANGLEUNIT["degree",0.0174532925199433]],
              AXIS["geodetic longitude (Lon)",east,
                    ORDER[2],
                    ANGLEUNIT["degree",0.0174532925199433]],
         ID["EPSG",4269]]
    Data axis to CRS axis mapping: 2,1
    ```
    - The coordinate system is NAD83 (EPSG:4269), which uses the North American Datum 1983.

5. **Origin and Pixel Size**:
    ```
    Origin = (-97.999999407110110,31.000000503864648)
    Pixel Size = (0.000277777777778,-0.000277777777778)
    ```
    - The origin (upper-left corner) of the raster is at coordinates (-97.9999994, 31.0000005).
    - Each pixel represents approximately 0.00027778 degrees.

6. **Metadata**:
    ```
    Metadata:
      AREA_OR_POINT=Point
    ```
    - The metadata indicates that the data represents points.

7. **Corner Coordinates**:
    ```
    Corner Coordinates:
    Upper Left  ( -97.9999994,  31.0000005) ( 98d 0' 0.00"W, 31d 0' 0.00"N)
    Lower Left  ( -97.9999994,  30.0000005) ( 98d 0' 0.00"W, 30d 0' 0.00"N)
    Upper Right ( -96.9999994,  31.0000005) ( 97d 0' 0.00"W, 31d 0' 0.00"N)
    Lower Right ( -96.9999994,  30.0000005) ( 97d 0' 0.00"W, 30d 0' 0.00"N)
    Center      ( -97.4999994,  30.5000005) ( 97d30' 0.00"W, 30d30' 0.00"N)
    ```
    - The coordinates of the four corners and the center of the raster are provided in both decimal degrees and degrees, minutes, seconds format.

8. **Band Information**:
    ```
    Band 1 Block=3600x3600 Type=Float32, ColorInterp=Undefined
      Min=79.900 Max=398.900 
      Minimum=79.900, Maximum=398.900, Mean=193.121, StdDev=63.589
      NoData Value=-32767
      Unit Type: m
      Metadata:
         STATISTICS_MAXIMUM=398.89999389648
         STATISTICS_MEAN=193.12105660542
         STATISTICS_MINIMUM=79.900001525879
         STATISTICS_STDDEV=63.588582941289
    ```
    - The raster has one band (Band 1) with a block size of 3600x3600 pixels.
    - The data type is `Float32`.
    - The minimum and maximum values in the band are 79.900 and 398.900, respectively.
    - The mean value is 193.121, and the standard deviation is 63.589.
    - The NoData value is -32767, indicating missing or undefined data.
    - The unit type is meters (m).
    - Additional statistics are provided in the metadata.
```

In [ ]:
# Use the gdalinfo command to get information about the raster file 'doqq.tif'
# This command provides detailed metadata about the raster file, such as its size, coordinate system, and more
!gdalinfo ./data/doqq.tif

### Getting JSON formatted metadata


### Why Getting Metadata Formatted as JSON from GDAL/OGR is Useful

Obtaining metadata in JSON format from GDAL/OGR offers several advantages:

1. **Structured Data**: JSON provides a structured format that is easy to parse and manipulate programmatically. This structure allows for straightforward extraction and utilization of specific metadata elements.

2. **Interoperability**: JSON is a widely accepted data interchange format, making it compatible with various programming languages and tools. This interoperability facilitates seamless integration into diverse workflows and systems.

3. **Readability**: JSON is human-readable, making it easier for developers and analysts to understand the metadata without requiring specialized knowledge of the data format.

4. **Automation**: JSON's structured nature enables automated processing and analysis. Scripts and applications can easily parse JSON to extract relevant metadata, enabling automated workflows and data processing pipelines.

5. **Flexibility**: JSON can represent complex nested structures, allowing for detailed and comprehensive metadata representation. This flexibility ensures that all relevant information about the dataset is captured and accessible.

6. **Standardization**: Using JSON for metadata ensures a standardized format across different datasets and tools. This standardization simplifies data management and reduces the risk of errors or inconsistencies.

By leveraging JSON for metadata extraction, users can enhance their data processing capabilities, improve interoperability, and streamline their workflows.


In [ ]:
# Use the gdalinfo command to get information about the DEM file in JSON format
# The DEM file is located in the 'dem10m' directory within the 'data' directory
!gdalinfo -json ./data/dem10m/dem10m.dem

### Explanation of `gdalinfo` Output for `doqq.tif` Raster Dataset

The output of the `gdalinfo` command provides detailed metadata about the `doqq.tif` raster dataset. Here's a breakdown of the key components:

1. **Driver**:
    ```
    Driver: GTiff/GeoTIFF
    ```
    - The driver used to read the file is `GTiff`, which supports the GeoTIFF format.

2. **Files**:
    ```
    Files: ./data/doqq.tif
    ```
    - The primary file is `doqq.tif`.

3. **Size**:
    ```
    Size is 2394, 2344
    ```
    - The raster size is 2394 pixels by 2344 pixels.

4. **Coordinate System**:
    ```
    Coordinate System is:
    PROJCRS["NAD83 / UTM zone 14N",
        BASEGEOGCRS["NAD83",
            DATUM["North American Datum 1983",
                ELLIPSOID["GRS 1980",6378137,298.257222101,
                    LENGTHUNIT["metre",1]]],
            PRIMEM["Greenwich",0,
                ANGLEUNIT["degree",0.0174532925199433]],
            ID["EPSG",4269]],
        CONVERSION["UTM zone 14N",
            METHOD["Transverse Mercator",
                ID["EPSG",9807]],
            PARAMETER["Latitude of natural origin",0,
                ANGLEUNIT["degree",0.0174532925199433],
                ID["EPSG",8801]],
            PARAMETER["Longitude of natural origin",-99,
                ANGLEUNIT["degree",0.0174532925199433],
                ID["EPSG",8802]],
            PARAMETER["Scale factor at natural origin",0.9996,
                SCALEUNIT["unity",1],
                ID["EPSG",8805]],
            PARAMETER["False easting",500000,
                LENGTHUNIT["metre",1],
                ID["EPSG",8806]],
            PARAMETER["False northing",0,
                LENGTHUNIT["metre",1],
                ID["EPSG",8807]]],
        CS[Cartesian,2],
            AXIS["(E)",east,
                ORDER[1],
                LENGTHUNIT["metre",1]],
            AXIS["(N)",north,
                ORDER[2],
                LENGTHUNIT["metre",1]],
        USAGE[
            SCOPE["Engineering survey, topographic mapping."],
            AREA["North America - between 102°W and 96°W - onshore and offshore. Canada - Manitoba; Nunavut; Saskatchewan. United States (USA) - Iowa; Kansas; Minnesota; Nebraska; North Dakota; Oklahoma; South Dakota; Texas."],
            BBOX[25.83,-102,84,-96]],
        ID["EPSG",26914]]
    Data axis to CRS axis mapping: 1,2
    ```
    - The coordinate system is NAD83 / UTM zone 14N (EPSG:26914), which uses the North American Datum 1983.

5. **Origin and Pixel Size**:
    ```
    Origin = (621918.000000000000000,3356441.000000000000000)
    Pixel Size = (0.500000000000000,-0.500000000000000)
    ```
    - The origin (upper-left corner) of the raster is at coordinates (621918, 3356441).
    - Each pixel represents 0.5 meters.

6. **Metadata**:
    ```
    Metadata:
      AREA_OR_POINT=Area
    ```
    - The metadata indicates that the data represents an area.

7. **Image Structure Metadata**:
    ```
    Image Structure Metadata:
      INTERLEAVE=PIXEL
    ```
    - The image is interleaved by pixel.

8. **Corner Coordinates**:
    ```
    Corner Coordinates:
    Upper Left  (  621918.000, 3356441.000) ( 97d43'54.13"W, 30d20' 1.21"N)
    Lower Left  (  621918.000, 3355269.000) ( 97d43'54.62"W, 30d19'23.14"N)
    Upper Right (  623115.000, 3356441.000) ( 97d43' 9.31"W, 30d20' 0.77"N)
    Lower Right (  623115.000, 3355269.000) ( 97d43' 9.80"W, 30d19'22.70"N)
    Center      (  622516.500, 3355855.000) ( 97d43'31.97"W, 30d19'41.96"N)
    ```
    - The coordinates of the four corners and the center of the raster are provided in both UTM and degrees, minutes, seconds format.

9. **Bands**:
    ```
    Band 1 Block=2394x1 Type=Byte, ColorInterp=Red
      Mask Flags: PER_DATASET ALPHA 
    Band 2 Block=2394x1 Type=Byte, ColorInterp=Green
      Mask Flags: PER_DATASET ALPHA 
    Band 3 Block=2394x1 Type=Byte, ColorInterp=Blue
      Mask Flags: PER_DATASET ALPHA 
    Band 4 Block=2394x1 Type=Byte, ColorInterp=Alpha
    ```
    - The raster has four bands:
        - Band 1: Red channel
        - Band 2: Green channel
        - Band 3: Blue channel
        - Band 4: Alpha channel
    - Each band is of type `Byte` and has a block size of 2394x1 pixels.
    - The alpha mask is applied per dataset.

## Converting Data

### Exporting original data into new formats

GDAL (Geospatial Data Abstraction Library) and OGR (part of the GDAL library for vector data) are powerful tools for converting between various geospatial data formats. These tools support a wide range of formats, making it easy to transform data to meet the requirements of different applications and workflows.

### Key Benefits of Using GDAL/OGR for Data Conversion

1. **Support for Multiple Formats**: GDAL/OGR supports numerous raster and vector formats, including GeoTIFF, PNG, JPEG, Shapefile, GeoJSON, and more. This versatility allows users to work with various data sources seamlessly.

2. **Efficient Data Processing**: GDAL/OGR is optimized for performance, enabling efficient conversion of large datasets. This is crucial for handling high-resolution imagery and extensive geospatial data.

3. **Command-Line and Programmatic Access**: GDAL/OGR provides both command-line utilities (e.g., `gdal_translate`, `ogr2ogr`) and programmatic interfaces (e.g., Python bindings) for data conversion. This flexibility allows users to integrate data conversion into automated workflows and scripts.

4. **Reprojection and Transformation**: GDAL/OGR can reproject and transform data to different coordinate systems during the conversion process. This is essential for integrating datasets from various sources with different spatial references.

5. **Data Subsetting and Clipping**: GDAL/OGR allows for subsetting and clipping data based on specified boundaries or windows during conversion. This functionality is useful for focusing analysis on specific areas of interest.

6. **Metadata Preservation**: GDAL/OGR ensures that essential metadata, such as coordinate systems and attribute information, is preserved during the conversion process. This is vital for maintaining the integrity and usability of the data.

By leveraging GDAL/OGR's capabilities for data conversion, users can efficiently manage, transform, and integrate geospatial data, making it an indispensable tool in the geospatial domain.

### What formats does GDAL/OGR support?

In [ ]:
# List available formats supported by OGR (used for vector data)
!ogr2ogr --formats

# List available formats supported by GDAL (used for raster data)
!gdal_translate --formats

### shapefile → geojson

Vector data conversion is a fundamental process in Geographic Information Systems (GIS) that involves transforming spatial data from one format to another. Two common formats for vector data are Shapefile (SHP) and GeoJSON. Understanding the utility of converting between these formats is essential for effective data management and interoperability in GIS applications.

### Data Conversion with OGR for Vector Data Formats

#### Shapefile (SHP)
Shapefile is a popular vector data format developed by Esri for geographic information system (GIS) software. It is widely used for storing geometric location and associated attribute information. A Shapefile is actually a collection of files with the same base name but different extensions, such as `.shp` (geometry), `.shx` (shape index), and `.dbf` (attribute data).

#### GeoJSON
GeoJSON is an open standard format designed for representing simple geographical features, along with their non-spatial attributes, using JavaScript Object Notation (JSON). GeoJSON supports various geometry types like Point, LineString, Polygon, and MultiPolygon, making it a versatile format for web-based mapping and data sharing.

### Utility of Converting Shapefile to GeoJSON

1. **Interoperability**: GeoJSON is widely supported by web mapping libraries such as Leaflet and Mapbox, making it easier to integrate spatial data into web applications. Converting Shapefiles to GeoJSON enhances the ability to share and visualize data across different platforms and technologies.

2. **Simplicity and Readability**: GeoJSON is a text-based format that is human-readable and easy to understand. This simplicity makes it easier for developers to manipulate and debug spatial data compared to the binary format of Shapefiles.

3. **Web Compatibility**: GeoJSON is inherently compatible with web technologies, allowing for seamless integration with APIs and web services. This makes it an ideal format for web-based GIS applications and interactive maps.

4. **Data Size**: GeoJSON files are often smaller in size compared to Shapefiles because they do not require multiple accompanying files. This can be advantageous for data transfer and storage, especially in web applications where bandwidth and speed are critical.

5. **Flexibility**: GeoJSON can be easily extended with additional properties and metadata, providing greater flexibility in how spatial data is represented and used.

### Conclusion

Converting Shapefiles to GeoJSON is a valuable process in the realm of GIS, offering numerous benefits in terms of interoperability, simplicity, web compatibility, data size, and flexibility. Understanding these advantages helps GIS professionals and developers make informed decisions about the best formats to use for their spatial data needs.

In [ ]:
# Convert the shapefile to GeoJSON format
!ogr2ogr -f geojson ./out/zipcodes.geojson ./data/zipcodes/zipcodes.shp

# Display summary information about the 'zipcodes' layer in the newly created GeoJSON file
!ogrinfo -so ./out/zipcodes.geojson zipcodes

### GeoTIFF → georeferenced PNG
## Data Conversion with GDAL for Raster Data

GDAL (Geospatial Data Abstraction Library) is a powerful open-source library for reading, writing, and manipulating raster and vector geospatial data. One of the key functionalities of GDAL is its ability to convert data between different formats, which is essential for various geospatial workflows and applications.

### Understanding Raster Data Formats: GeoTIFF vs. PNG

#### GeoTIFF
GeoTIFF is a widely used raster data format that includes georeferencing information, allowing the data to be accurately placed in geographic space. It supports various types of data, including satellite imagery, digital elevation models (DEMs), and other geospatial datasets. Key features of GeoTIFF include:

- **Georeferencing**: GeoTIFF files contain metadata that describes the spatial reference system, allowing the data to be accurately mapped to real-world coordinates.
- **Multi-band Support**: GeoTIFF can store multiple bands of data, such as different spectral bands of satellite imagery.
- **Data Precision**: GeoTIFF supports various data types, including integer and floating-point values, making it suitable for scientific and analytical purposes.

#### PNG
PNG (Portable Network Graphics) is a raster image format commonly used for web graphics and visualization. While it does not inherently support georeferencing, it can be used to display geospatial data when combined with external georeferencing information. Key features of PNG include:

- **Visualization**: PNG is ideal for creating visual representations of geospatial data, such as maps and overlays, due to its support for transparency and lossless compression.
- **Simplicity**: PNG files are easy to share and display on various platforms, making them suitable for web applications and presentations.
- **Limited Geospatial Use**: Unlike GeoTIFF, PNG does not store georeferencing information within the file itself, which limits its use for precise spatial analysis. Instead, georeferencing information for PNG files is often stored in a separate "sidecar" file, such as a world file (.wld), which contains the necessary spatial reference data to align the image correctly in geographic space.

### Data vs. Visualization

- **Data (GeoTIFF)**: GeoTIFF is primarily used for storing and analyzing geospatial data. It retains the full precision and georeferencing information necessary for accurate spatial analysis and integration with other geospatial datasets.
- **Visualization (PNG)**: PNG is used for visualizing geospatial data, providing a clear and visually appealing representation of the data. It is suitable for creating maps and graphics for presentations, reports, and web applications.

### Conclusion

Understanding the differences between GeoTIFF and PNG formats is crucial for effectively managing and utilizing raster data in geospatial workflows. While GeoTIFF is ideal for data storage and analysis, PNG excels in visualization and presentation. GDAL's ability to convert between these formats allows users to leverage the strengths of each format for their specific needs.

In [ ]:
# Convert the GeoTIFF file 'doqq.tif' to a PNG file format
!gdal_translate -of png ./data/doqq.tif ./out/converted_doqq.png

# Get information about the newly created PNG file
!gdalinfo ./out/converted_doqq.png

In [ ]:
# Create a map centered around the approximate location of the raster data
map_center = [30.2672, -97.7431]  # Coordinates for Austin, TX
m = leafmap.Map(center=map_center, zoom=10)

# Define the path to the converted PNG file
converted_doqq_path = './out/converted_doqq.png'

# Add the raster layer to the map
m.add_raster(converted_doqq_path, layer_name="Converted DOQQ")

# Display the map
m

## Reprojecting Data with GDAL/OGR

### Reprojecting Data Using EPSG Codes

Reprojecting data involves transforming spatial data from one coordinate reference system (CRS) to another. This process is essential in Geographic Information Systems (GIS) to ensure that datasets with different projections can be accurately overlaid and analyzed together. GDAL (Geospatial Data Abstraction Library) and OGR (part of the GDAL library for vector data) provide powerful tools for reprojecting both raster and vector data.

### Why Projection is Necessary for Some Tasks

1. **Data Integration**: Different datasets may come in various coordinate systems. Reprojecting them to a common CRS ensures that they align correctly on a map, allowing for accurate spatial analysis and visualization.

2. **Accuracy**: Some projections are better suited for specific regions or types of analysis. For example, UTM (Universal Transverse Mercator) projections minimize distortion over small areas, making them ideal for detailed local studies.

3. **Interoperability**: Many GIS applications and web mapping services require data to be in a specific projection. Reprojecting data ensures compatibility with these systems, facilitating data sharing and integration.

4. **Analysis**: Certain spatial analyses, such as distance calculations and area measurements, require data to be in a projection that preserves these properties accurately. Reprojecting data to an appropriate CRS ensures the validity of such analyses.

5. **Visualization**: Different projections can be used to emphasize specific aspects of the data. For example, conformal projections preserve shape, while equal-area projections preserve area, each serving different visualization needs.

By leveraging GDAL/OGR's capabilities for reprojecting data, users can ensure that their spatial datasets are accurately aligned, compatible with various systems, and suitable for precise analysis and visualization.

In [ ]:
!ogrinfo -so -al ./data/pools/pools.shp  # Display summary information about all layers in the 'pools.shp' shapefile


In [ ]:
# Reproject the 'pools' shapefile to EPSG:4326 (WGS 84) coordinate system and save it as 'reprojected_pools.shp'
!ogr2ogr -t_srs EPSG:4326 ./out/reprojected_pools.shp ./data/pools/pools.shp

In [ ]:
!ogrinfo -so ./out/reprojected_pools.shp reprojected_pools # Display summary information about the 'reprojected_pools' layer

In [ ]:
# Get the original CRS of the TIFF file using gdalinfo with the -proj4 option
!gdalinfo -proj4 ./data/doqq.tif

In [ ]:
# Reproject the 'doqq.tif' raster file to EPSG:4326 (WGS 84) coordinate system and save it as 'reprojected_doqq.tif'
!gdalwarp -t_srs EPSG:4326 ./data/doqq.tif ./out/reprojected_doqq.tif

In [ ]:
!gdalinfo ./out/reprojected_doqq.tif  # Display summary information about the 'reprojected_doqq' raster    

## Querying Data

### Query data using SQL with GDAL

GDAL/OGR's SQL dialect provides a powerful way to query and manipulate geospatial datasets. By using SQL commands, users can perform complex queries, filter data, and create subsets of larger datasets. This functionality is particularly useful for workflows that require specific data extraction or analysis.

### Key Features of GDAL/OGR's SQL Dialect

1. **Subsetting Data**: SQL queries can be used to select specific features or attributes from a dataset, allowing users to create subsets of data that meet certain criteria.
2. **Filtering**: Users can apply filters to datasets to extract only the relevant information, such as selecting features within a certain geographic area or with specific attribute values.
3. **Joining Datasets**: SQL can be used to join multiple datasets based on common attributes, enabling more complex data analysis and integration.
4. **Aggregation**: SQL queries can perform aggregation operations, such as counting features, calculating averages, or summarizing data based on specific attributes.
5. **Creating New Datasets**: The results of SQL queries can be saved as new datasets, making it easy to generate new files based on the query results.

By leveraging GDAL/OGR's SQL capabilities, users can streamline their geospatial data processing workflows, making it easier to manage, analyze, and visualize large and complex datasets.

### `COUNT()`

In [ ]:
# Display the total count of features in the 'reprojected_pools' shapefile
!ogrinfo ./out/reprojected_pools.shp -sql "SELECT COUNT(*) FROM reprojected_pools"

### `WHERE` clauses

In [ ]:
# Display detailed information about the feature with FID (Feature ID) 1 in the 'reprojected_pools' shapefile
!ogrinfo -q ./out/reprojected_pools.shp -sql "SELECT * FROM reprojected_pools WHERE fid = 1"



### Combining 

In [ ]:
# Display the count of features in the 'reprojected_pools' shapefile where the 'FEATURE' attribute is 'Above ground'
!ogrinfo -q ./out/reprojected_pools.shp -sql "SELECT COUNT(*) FROM reprojected_pools WHERE FEATURE = 'Above ground'"

### Saving query results to a new file

Create new files based on SQL queries

In [ ]:
# Select zip codes for Austin and save to a new shapefile
!ogr2ogr ./out/austin_zips.shp ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE name = 'Austin'"

### Visualize the Query Result, with 'leafmap'

In [ ]:
# Load the original shapefile using geopandas
original_gdf = gpd.read_file('./data/zipcodes/zipcodes.shp')

# Load the query result shapefile using geopandas
austin_zips_gdf = gpd.read_file('./out/austin_zips.shp')

# Create a map centered around the data
austin_zip_map = leafmap.Map(center=[austin_zips_gdf.geometry.centroid.y.mean(), austin_zips_gdf.geometry.centroid.x.mean()], zoom=10)

# Add the original shapefile layer to the map in grey
austin_zip_map.add_gdf(original_gdf, layer_name="Original Zipcodes", style={"color": "grey", "weight": 1})

# Add the query result shapefile layer to the map in red
austin_zip_map.add_gdf(austin_zips_gdf, layer_name="Austin Zipcodes", style={"color": "red", "weight": 2})

# Display the map
austin_zip_map

In [ ]:
# Select zip codes with SHAPE_AREA less than 10,000,000 and save to a new shapefile
!ogr2ogr ./out/under_10mil.shp ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE SHAPE_AREA < 10000000"

In [ ]:
# Load the shapefile using geopandas
under_10mil_gdf = gpd.read_file('./out/under_10mil.shp')

# Create a map centered around the data
under_10mil_map = leafmap.Map(center=[under_10mil_gdf.geometry.centroid.y.mean(), under_10mil_gdf.geometry.centroid.x.mean()], zoom=10)

# Add the original zipcodes shapefile layer to the map in grey
under_10mil_map.add_gdf(original_gdf, layer_name="Original Zipcodes", style={"color": "grey", "weight": 1})

# Add the under_10mil shapefile layer to the map in red
under_10mil_map.add_gdf(under_10mil_gdf, layer_name="Under 10 Million Shape Area", style={"color": "red", "weight": 2})

# Display the map
under_10mil_map

In [ ]:
# Convert the shapefile to GeoJSON format for the ZIP code '78756'
!ogr2ogr -f geojson ./out/78756.geojson ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE ZIPCODE = '78756'"

In [ ]:
# Load the geojson file using geopandas
gdf_78756 = gpd.read_file('./out/78756.geojson')

# Create a map centered around the data
map_78756 = leafmap.Map(center=[gdf_78756.geometry.centroid.y.mean(), gdf_78756.geometry.centroid.x.mean()], zoom=12)

# Add the geojson layer to the map
map_78756.add_gdf(gdf_78756, layer_name="78756", style={"color": "red", "weight": 2})

# Display the map
map_78756

## Clipping to create new data
## Introduction to Using GDAL/OGR for Clipping Datasets and Selecting by Location

### Overview

GDAL (Geospatial Data Abstraction Library) and OGR (part of the GDAL library for vector data) are powerful open-source tools for manipulating and analyzing geospatial data. One of the key functionalities provided by GDAL/OGR is the ability to clip datasets and perform spatial queries, such as selecting features by location. These operations are essential for focusing on specific areas of interest within larger datasets, enabling more efficient data analysis and visualization.

### Clipping Datasets

Clipping is the process of extracting a subset of a dataset based on a defined boundary. This boundary can be a polygon, a shapefile, or any other vector data format that defines the area of interest. Clipping is particularly useful when working with large datasets, as it allows users to isolate and work with only the relevant portions of the data.

#### Key Benefits of Clipping:
1. **Focus on Areas of Interest**: Clipping allows users to concentrate on specific geographic areas, making it easier to analyze and visualize data relevant to a particular region.
2. **Reduce Data Size**: By extracting only the necessary data, clipping reduces the size of the dataset, leading to more efficient processing and storage.
3. **Improve Performance**: Smaller, clipped datasets can be processed and visualized more quickly, improving overall performance in GIS applications.


In [ ]:
# Use ogr2ogr to clip the 'reprojected_pools.shp' shapefile using the '78756.geojson' file as the clipping boundary
# The output will be saved as '78756_pools.shp' and will be all of the pools that fall within the 78756 zip code boundary
!ogr2ogr -clipsrc ./out/78756.geojson ./out/78756_pools.shp ./out/reprojected_pools.shp

!ogrinfo -so ./out/78756_pools.shp 78756_pools  # Display summary information about the '78756_pools' shapefile

### Visualize the subset of pools 

In [ ]:
# Load the shapefile using geopandas
pools_gdf = gpd.read_file('./out/78756_pools.shp')

# Create a map centered around the data with Google Satellite as the basemap
pools_map = leafmap.Map(center=[pools_gdf.geometry.centroid.y.mean(), pools_gdf.geometry.centroid.x.mean()], zoom=14)
pools_map.add_basemap("SATELLITE")

# Add the shapefile layer to the map with increased outline width
pools_map.add_gdf(pools_gdf, layer_name="78756 Pools", style={"weight": 5})  # Increase the weight to 5

# Display the map
pools_map

### Clipping raster data with vector data

## Clipping Raster Data with Vector Data Using `gdalwarp`

Clipping raster data with vector data is a common geospatial operation that involves extracting a subset of a raster dataset based on the boundaries defined by a vector dataset. This process is essential for focusing analysis on specific areas of interest, reducing data size, and improving processing efficiency. GDAL (Geospatial Data Abstraction Library) provides a powerful tool called `gdalwarp` for performing this operation.

### Why Clip Raster Data?

1. **Focus on Areas of Interest**: Clipping allows users to isolate and analyze specific geographic regions within a larger raster dataset, making it easier to focus on areas of interest.
2. **Reduce Data Size**: By extracting only the necessary data, clipping reduces the size of the dataset, leading to more efficient storage and processing.
3. **Improve Performance**: Smaller, clipped datasets can be processed and visualized more quickly, improving overall performance in GIS applications.
4. **Data Preparation**: Clipping is often a necessary step in preparing data for further analysis, visualization, or integration with other datasets.

### Using `gdalwarp` for Clipping

`gdalwarp` is a versatile command-line tool provided by GDAL for warping (reprojecting) and resampling raster data. It also supports clipping raster datasets using vector data as a mask. The basic syntax for clipping a raster with `gdalwarp` is as follows:

```sh
gdalwarp -cutline <vector_file> -crop_to_cutline <input_raster> <output_raster>
```

- **`-cutline <vector_file>`**: Specifies the vector file (e.g., shapefile, GeoJSON) that defines the clipping boundary.
- **`-crop_to_cutline`**: Ensures that the output raster is cropped to the extent of the cutline.
- **`<input_raster>`**: The input raster file to be clipped.
- **`<output_raster>`**: The output raster file that will contain the clipped data.

### Example

Suppose we have a raster file `input.tif` and a vector file `boundary.shp` that defines the area of interest. The following command clips the raster using the vector boundary:

```sh
gdalwarp -cutline boundary.shp -crop_to_cutline input.tif output_clipped.tif
```

This command will create a new raster file `output_clipped.tif` that contains only the data within the boundaries defined by `boundary.shp`.

### Additional Options

`gdalwarp` offers several additional options that can be useful when clipping raster data:

- **`-dstalpha`**: Adds an alpha band to the output raster to handle transparency.
- **`-of <format>`**: Specifies the output format (e.g., GTiff for GeoTIFF).
- **`-t_srs <srs>`**: Reprojects the output raster to a specified spatial reference system.


In [ ]:
# Use gdalwarp to clip the 'doqq.tif' raster file using the '78756.geojson' file as the clipping boundary
# -cutline: specifies the vector file to use for clipping
# -dstalpha: adds an alpha band to the output image to handle transparency
# The output will be saved as '78756_doqq.tif'

!gdalwarp -cutline ./out/78756.geojson -dstalpha ./data/doqq.tif ./out/78756_doqq.tif

!gdalinfo ./out/78756_doqq.tif  # Display summary information about the '78756_doqq' raster

In [ ]:
# Create a map centered around the approximate location of the raster data
doqq_map_center = [30.2672, -97.7431]  # Coordinates for Austin, TX
doqq_map = leafmap.Map(center=doqq_map_center, zoom=10)

# Define the path to the 78756_doqq.tif file
doqq_tif_path = './out/78756_doqq.tif'

# Add the raster layer to the map
doqq_map.add_raster(doqq_tif_path, layer_name="78756 DOQQ")

# Display the map
doqq_map

In [ ]:
# Use gdal_translate to clip a portion of the DEM file
# -srcwin: specifies the window of pixels to extract (xoff, yoff, xsize, ysize)
# -of: specifies the output format (USGSDEM in this case)
# The output will be saved as 'clipped_dem.dem'

!gdal_translate -srcwin 0 0 1000 1000 -of USGSDEM ./data/dem10m/dem10m.dem ./out/clipped_dem.dem

!gdalinfo ./out/clipped_dem.dem  # Display summary information about the 'clipped_dem' raster

In [ ]:
# Create a map centered around the approximate location of the raster data
clipped_dem_map_center = [30.2672, -97.7431]  # Coordinates for Austin, TX
clipped_dem_map = leafmap.Map(center=clipped_dem_map_center, zoom=13)

# Define the path to the clipped DEM file
clipped_dem_path = './out/clipped_dem.dem'

# Add the raster layer to the map
clipped_dem_map.add_raster(clipped_dem_path, layer_name="Clipped DEM")

# Display the map
clipped_dem_map

## Fun with GDAL!

In [ ]:
# Use gdaldem to generate a hillshade from the DEM file
# -of: specifies the output format (PNG in this case)
# The output will be saved as 'hillshade.png'

!gdaldem hillshade -of png ./out/clipped_dem.dem ./out/hillshade.png

!gdalinfo ./out/hillshade.png  # Display summary information about the 'hillshade' raster

### Visualize our (ugly) hillshade image

In [ ]:
# Create a map centered around the approximate location of the hillshade data
hillshade_map_center = [30.2672, -97.7431]  # Coordinates for Austin, TX
hillshade_map = leafmap.Map(center=hillshade_map_center, zoom=10)

# Define the path to the hillshade PNG file
hillshade_path = './out/hillshade.png'

# Add the hillshade raster layer to the map
hillshade_map.add_raster(hillshade_path, layer_name="Hillshade")

# Display the map
hillshade_map

## Create a (better) hillshade with GDAL and Z-factor
### Why does the hillshade look dark and unappealing without the use of a Z-factor?

When generating a hillshade, the Z-factor is a crucial parameter that adjusts the vertical exaggeration of the elevation data. Without applying a Z-factor, the elevation values are interpreted directly, which can lead to a hillshade that appears dark and lacks contrast. This is especially true if the elevation values are in meters but the horizontal units are in degrees (common in geographic coordinate systems). The mismatch in units causes the elevation changes to appear less pronounced, resulting in a flat and unappealing hillshade.

### Calculation of Z-factor

The Z-factor is used to convert elevation units to match the horizontal units. For more information on determining a Z-factor, refer to this [ESRI blog post](https://www.esri.com/arcgis-blog/products/arcgis-desktop/defense/determining-a-z-factor-for-scaling-linear-elevation-units-to-match-geographic-coordinate-values/).


### Calculate and apply Z-factor when creating a hillshade

In [ ]:
import math

# Calculate the latitude of the center of the DEM
latitude = 30.2672  # Example latitude for Austin, TX

# Calculate the z-factor using the formula: z_factor = 1 / (111320 * cos(latitude * pi / 180))
z_factor = 1 / (111320 * math.cos(latitude * math.pi / 180))

# Use gdaldem to generate a hillshade from the DEM file with the calculated z-factor
# -of: specifies the output format (PNG in this case)
# The output will be saved as 'hillshade_with_zfactor.png'

!gdaldem hillshade -z {z_factor} -of png ./out/clipped_dem.dem ./out/hillshade_with_zfactor.png

!gdalinfo ./out/hillshade_with_zfactor.png  # Display summary information about the 'hillshade_with_zfactor' raster

## Visualize the new hillshade layer to see the difference

In [ ]:
# Create a split-panel map with the specified layers
hillshade_swipe_map = leafmap.split_map(
    left_layer="./out/hillshade.png", 
    right_layer="./out/hillshade_with_zfactor.png"
)

# Display the map
hillshade_swipe_map


## Create a Color Relief from a DEM using a Color Ramp File

### Overview

Creating a color relief involves mapping elevation values from a DEM to specific colors defined in a color ramp file. This process enhances the visual representation of the terrain by applying colors that correspond to different elevation ranges.

### Steps to Create a Color Relief

1. **Load the DEM**: Read the DEM data, which contains elevation values for each pixel.
2. **Define the Color Ramp**: Create a color ramp file (`colorramp.txt`) that specifies which colors correspond to which elevation ranges.
3. **Apply the Color Ramp**: Use a tool (like GDAL) to apply the color ramp to the DEM, generating a color relief image.

### Structure of `colorramp.txt`

The `colorramp.txt` file typically contains lines where each line defines a mapping from an elevation value to a color. The format is usually:

`elevation_value red green blue`



- `elevation_value`: The elevation at which the color should be applied.
- `red`, `green`, `blue`: The RGB color values (0-255) to be applied at that elevation.

#### Example of `colorramp.txt`



In [ ]:
0 0 0 255       # Blue for sea level
100 0 255 0     # Green for low elevations
500 255 255 0   # Yellow for mid elevations
1000 255 0 0    # Red for high elevations
2000 255 255 255 # White for very high elevations

### Applying the Color Ramp to the DEM

### Explanation of the Process

1. **Loading the DEM**: The DEM file (`input_dem.tif`) is loaded, containing elevation data for each pixel.
2. **Reading the Color Ramp**: The `colorramp.txt` file is read, and the elevation-to-color mappings are stored.
3. **Generating the Color Relief**:
   - For each pixel in the DEM, the elevation value is retrieved.
   - The corresponding color is determined by finding the closest elevation value in the color ramp file.
   - The pixel is colored with the RGB values specified in the color ramp file.
4. **Output**: The resulting image (`output_color_relief.tif`) is saved, showing the terrain with the applied color relief.

In [ ]:
# Define the path to the DEM file and the color ramp file
dem_file = './out/clipped_dem.dem'
color_ramp_file = './data/colorramp.txt'
output_color_relief = './out/colorrelief.tif'

# Use gdaldem to generate a color relief from the DEM file using the color ramp file
!gdaldem color-relief {dem_file} {color_ramp_file} {output_color_relief}

!gdalinfo {output_color_relief}  # Display summary information about the 'colorrelief' raster

### Visualize the Color Relief

In [ ]:
# Create a map centered around the approximate location of the color relief data
color_relief_map_center = [30.2672, -97.7431]  # Coordinates for Austin, TX
color_relief_map = leafmap.Map(center=color_relief_map_center, zoom=10)

# Define the path to the color relief TIFF file
color_relief_path = './out/colorrelief.tif'

# Add the color relief raster layer to the map
color_relief_map.add_raster(color_relief_path, layer_name="Color Relief")

# Display the map
color_relief_map

## Vectorization of Raster Data

### Overview

Vectorization is the process of converting raster data into vector data. Raster data consists of a grid of cells or pixels, each with a specific value representing information such as elevation, temperature, or land cover. Vector data, on the other hand, represents geographic features using points, lines, and polygons. Vectorization is essential for various geospatial analyses and applications, as it allows for more precise and flexible manipulation of spatial data.

### Why Vectorize Raster Data?

1. **Precision**: Vector data provides more precise representations of geographic features compared to raster data, which is limited by the resolution of the grid cells.
2. **Data Integration**: Vector data can be easily integrated with other vector datasets, enabling more complex spatial analyses and operations.
3. **Storage Efficiency**: Vector data can be more storage-efficient than raster data, especially for representing linear and polygonal features.
4. **Enhanced Analysis**: Vector data allows for advanced spatial analyses, such as network analysis, proximity analysis, and topological operations.

### Common Vectorization Techniques

1. **Thresholding**: Converting raster cells that meet a certain condition (e.g., elevation above a specific value) into vector features.
2. **Edge Detection**: Identifying and vectorizing the boundaries of features within a raster dataset.
3. **Contour Extraction**: Generating contour lines from a raster dataset, representing lines of equal value (e.g., elevation).

## Creating Contour Lines from Raster Data

Contour lines are a common vector representation of raster data, particularly for elevation datasets. Contour lines connect points of equal value, providing a way to visualize the topography of an area. In this task, we will create contour lines from our `clipped_dem.dem` raster dataset.

### Steps to Create Contour Lines

1. **Load the DEM**: Read the Digital Elevation Model (DEM) data, which contains elevation values for each pixel.
2. **Define Contour Intervals**: Specify the intervals at which contour lines should be generated (e.g., every 20 meters).
3. **Generate Contour Lines**: Use a tool (like GDAL) to generate contour lines from the DEM, creating a vector dataset of lines representing equal elevation values.
4. **Save and Visualize**: Save the generated contour lines as a vector file (e.g., Shapefile) and visualize them on a map.

### Example Command

The following command uses `gdal_contour` to generate contour lines from the `clipped_dem.dem` file, with a contour interval of 20 units:


In [ ]:
# Create contour lines from a DEM
# -a elev: specifies the attribute name for elevation values
# -i 20: specifies the contour interval (20 units)
# clipped_dem.dem: input DEM file
# contours.shp: output shapefile for contours

!gdal_contour -a elev -i 20 ./out/clipped_dem.dem ./out/contours.shp

!ogrinfo -so ./out/contours.shp contours  # Display summary information about the 'contours' layer in the 'contours.shp' shapefile

In [ ]:
# Create a map centered around the approximate location of the color relief data
contour_map_center = [30.2672, -97.7431]  # Coordinates for Austin, TX
contour_map = leafmap.Map(center=contour_map_center, zoom=12)

# Define the path to the color relief TIFF file
color_relief_path = './out/colorrelief.tif'

# Define the path to the contours shapefile
contours_path = './out/contours.shp'

# Add the color relief raster layer to the map
contour_map.add_raster(color_relief_path, layer_name="Color Relief")

# Add the contours layer to the map with specified style
contour_map.add_vector(contours_path, layer_name="Contours", style={"color": "white", "weight": 0.5})

# Display the map
contour_map